<a href="https://colab.research.google.com/github/shaking54/CS114.K21.KHTN/blob/master/Sarcasm%20Detection/Sarcasm_Headlines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#*Sarcasm Dectection
https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection

- Input bài toán: Một chuỗi string

- Output: Chuỗi string nhận được có mang tính Sarcastic hay không

####*Thu thập dữ liệu

- Dữ liệu cũ về Sarcasm dectection được lấy từ Twitters. Tuy nhiên, do dữ liệu này có độ nhiễu nên vẫn còn hạn chế. Để vượt qua được điểm này thì dữ liệu mới về Sarcasm detection được lấy từ 2 websites đó là https://www.theonion.com/ và https://www.huffpost.com . Về dữ liệu mang tính Sarcastic thì được lấy từ trang web TheOnion ở trong mục News in Brief và News in Photos categories. Còn dữ liệu không mang tính Sarcastic thì được lấy từ trang web HuffPost.


In [1]:
!wget https://raw.githubusercontent.com/shaking54/CS114.K21.KHTN/master/Sarcasm%20Detection/Sarcasm_Headlines_Dataset.json

--2020-06-25 13:51:47--  https://raw.githubusercontent.com/shaking54/CS114.K21.KHTN/master/Sarcasm%20Detection/Sarcasm_Headlines_Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5616830 (5.4M) [text/plain]
Saving to: ‘Sarcasm_Headlines_Dataset.json’

Sarcasm_Headlines_D 100%[===================>]   5.36M  14.4MB/s    in 0.4s    

2020-06-25 13:51:48 (14.4 MB/s) - ‘Sarcasm_Headlines_Dataset.json’ saved [5616830/5616830]



In [2]:
import pandas as pd
data=pd.read_json('/content/Sarcasm_Headlines_Dataset.json', lines=True) #Load dataset from json file
data.head() #get 5 samples from dataset

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [3]:
sarcasm = data.drop(columns='article_link') # drop the column article_link

##*Xử lý dữ liệu

- Các headline sẽ được chuyển về lowercase, loại bỏ các ký tự đặc biệt.
- Sau đó sẽ phân chia dữ liệu train và dữ liệu test
- Khai báo Vectorize để biểu diễn dữ liệu train dưới dạng vector và tạo một từ điển từ dữ liệu train
- Sử dụng lại từ điển của dữ liệu trên để biểu diễn dữ liệu test dưới dạng vector

In [4]:
import re
def pre_process(headline):
	headline = headline.lower() # convert the headline to lower case                                      
	headline = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", headline) # remove 
	headline = re.sub("(\\d|\\WZhe Yang)+"," ", headline)                   
	return headline

sarcasm['headline'] = sarcasm['headline'].apply(lambda x:pre_process(x))
sarcasm['headline']

0        former versace store clerk sues over secret 'b...
1        the 'roseanne' revival catches up to our thorn...
2        mom starting to fear son's web series closest ...
3        boehner just wants wife to listen, not come up...
4        j.k. rowling wishes snape happy birthday in th...
                               ...                        
26704                 american politics in moral free-fall
26705                               america's best   hikes
26706                                reparations and obama
26707    israeli ban targeting boycott supporters raise...
26708                       gourmet gifts for the foodie  
Name: headline, Length: 26709, dtype: object

In [5]:
from sklearn.model_selection import train_test_split
X = sarcasm['headline']
y = sarcasm['is_sarcastic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3) #split train set and test set

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer1 = CountVectorizer(analyzer='word',ngram_range=(1,1), max_features=10000)
X_train = vectorizer1.fit_transform(X_train)

vectorizer2 = CountVectorizer(vocabulary=vectorizer1.vocabulary_,ngram_range=(1,1))
X_test = vectorizer2.fit_transform(X_test)

In [7]:
X_train.shape

(18696, 10000)

##Chọn Model để train dữ liệu
- Sử dụng các thuật toán như Suport vector classifications, Decision Tree classifications, Random Forest Classification, Voting Classification.


In [8]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
SVClassifiers = SVC()
SVClassifiers.fit(X_train, y_train, )
SVC_predict = SVClassifiers.predict(X_test)
SCV_score = accuracy_score(y_test, SVC_predict)
SCV_score

0.8451266691626108

In [9]:
from sklearn.svm import LinearSVC

LinearSVClassifier = LinearSVC()
LinearSVClassifier.fit(X_train, y_train)
Linear_predict = LinearSVClassifier.predict(X_test)
Linearscore = accuracy_score(y_test, Linear_predict)
Linearscore 

0.8133033820042431

In [10]:
from sklearn.tree import DecisionTreeClassifier

Decisionclassifier = DecisionTreeClassifier()
Decisionclassifier.fit(X_train, y_train)
Decisionclassifier_predict = Decisionclassifier.predict(X_test)
Decisionclassifier_score = accuracy_score(y_test, Decisionclassifier_predict)
Decisionclassifier_score

0.7541495070510421

In [11]:
from sklearn.ensemble import RandomForestClassifier
RFClassifier = RandomForestClassifier()
RFClassifier.fit(X_train, y_train)
RFClassifier_predict = RFClassifier.predict(X_test)
RFClassifier_score = accuracy_score(y_test, RFClassifier_predict)
RFClassifier_score
print(RFClassifier.score(X_test, y_test))

0.8124298015724448


In [12]:
# from sklearn.ensemble import VotingClassifier
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
# RFClassifier = RandomForestClassifier()
# SVClassifiers = SVC()
# Decisionclassifier = DecisionTreeClassifier()
# voting_clf = VotingClassifier(
#         estimators=[('svc', clf), ('rf', random_clf), ('dc', tree_clf)],
#         voting='hard'
# )

#* Xử lý dữ liệu mới.
### crawl dữ liệu mới từ 2 nguồn là Huffpost và TheOnion bằng scapper extension của google chrome
- Sau khi crawl dữ liệu thì được 2 file .txt
- Tiếp theo load 2 file .txt vừa crawl được lên google colab bằng pandas
- Gán labels cho dữ liệu từ Huffpost là 0 và TheOnion bằng 1
- Chuyển 2 bảng dữ liệu thành dataframe bằng pandas
- Gộp 2 bảng lại và lưu xuống file json


In [13]:
# import pandas as pd

# onion = pd.read_csv('newOnion.txt', sep='\n')
# onion['is_sarcastic'] = 1
# huff = pd.read_csv('Huffpost.txt', sep='\n')
# huff['is_sarcastic'] = 0
# onion = pd.DataFrame(data=onion)
# huff = pd.DataFrame(data=huff)
# onion = onion.append(huff, sort=1)
# onion.sample(frac=1)
# onion.to_json('headline_data.json', orient='records', lines=True)

#* Chạy Model đã train với dữ liệu mới tạo được
- Model Support vector machine classifications
    - performance được 79.2% so với trên dữ liệu test là xấp xỉ 84.5%

- Model Decision Tree classifications
    - Performance được 69.5 % so với xấp xỉ 75% với dữ liệu test

- Model Random Forest Classifications
    - Performance được 74.3 % so với xấp xỉ 81% trên dữ liệu test



In [14]:
!wget https://raw.githubusercontent.com/shaking54/CS114.K21.KHTN/master/Sarcasm%20Detection/headline_data.json

--2020-06-25 13:53:51--  https://raw.githubusercontent.com/shaking54/CS114.K21.KHTN/master/Sarcasm%20Detection/headline_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229984 (225K) [text/plain]
Saving to: ‘headline_data.json’

headline_data.json  100%[===================>] 224.59K  --.-KB/s    in 0.05s   

2020-06-25 13:53:51 (4.47 MB/s) - ‘headline_data.json’ saved [229984/229984]



In [15]:
newdata = pd.read_json('/content/headline_data.json', lines=True)
newdata.sample(frac=1)

,headline,is_sarcastic
1349,PM Defends Joining Huge Anti-Racism Protest De...,0
1085,Twitter Users Pile On Trump After He Asks If S...,0
808,Area Man Marries Woman He Barely Knows After 5...,1
624,Johnson & Johnson CEO Idly Wonders How Much Mo...,1
898,Desperate Boy Scouts Officials Announce New ‘Y...,1
...,...,...
887,John Bolton Asked To Leave Condo Board Meeting...,1
1937,John Oliver Wants Your Help ‘Making Tucker Car...,0
1163,The Democratic Party Launches Its First 2020 A...,0
178,"‘A Dash Of Soap Bubbles, 2 Pinches Of Sunshine...",1


In [16]:
newdata['headline'] = newdata['headline'].apply(lambda x:pre_process(x))
Xtest = newdata['headline']
ytest = newdata['is_sarcastic']

Xtest = vectorizer2.fit_transform(Xtest)

In [17]:
for cls in [SVClassifiers,LinearSVClassifier, Decisionclassifier,RFClassifier]:
    yscore = cls.score(Xtest, ytest)
    print(cls.__class__.__name__, yscore)

SVC 0.792
LinearSVC 0.7755
DecisionTreeClassifier 0.6905
RandomForestClassifier 0.7495


#*Sử dụng model để dự đoán một  đoạn headline bất kì

In [18]:
headline = input("Enter headline: ")
for cls in [SVClassifiers, LinearSVClassifier,Decisionclassifier,RFClassifier]:
    print(cls.__class__.__name__, cls.predict(vectorizer2.transform([pre_process(headline)])))

Enter headline: kèw
SVC [1]
LinearSVC [0]
DecisionTreeClassifier [1]
RandomForestClassifier [1]


#* Thử trên Data crawl được từ bạn Dương

In [19]:
!wget https://raw.githubusercontent.com/duongcscx/CS114.K21.KHTN/master/huffpost.json
!wget https://raw.githubusercontent.com/duongcscx/CS114.K21.KHTN/master/theonion.json

--2020-06-25 13:54:49--  https://raw.githubusercontent.com/duongcscx/CS114.K21.KHTN/master/huffpost.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160023 (156K) [text/plain]
Saving to: ‘huffpost.json’

huffpost.json       100%[===================>] 156.27K  --.-KB/s    in 0.04s   

2020-06-25 13:54:49 (3.66 MB/s) - ‘huffpost.json’ saved [160023/160023]

--2020-06-25 13:54:51--  https://raw.githubusercontent.com/duongcscx/CS114.K21.KHTN/master/theonion.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 309885 (303K) [text/plain]
Savi

In [20]:
#theonion = pd.read_json('/content/theonion.json', lines=True)
Duongdata = pd.read_json('/content/huffpost.json', lines=True)
Duongdata = pd.DataFrame(data=Duongdata)
#Duongdata = Duongdata.append(theonion)
Duongdata=Duongdata.drop(columns='article_link')


In [21]:
Duongdata.head()
len(Duongdata)

703

In [22]:
from sklearn.metrics import classification_report
Duongdata['headline'] = Duongdata['headline'].apply(lambda x:pre_process(x))
DuongX = Duongdata['headline']
Duongy = Duongdata['is_sarcastic']

DuongX = vectorizer2.fit_transform(DuongX)
for cls in [SVClassifiers,LinearSVClassifier, Decisionclassifier,RFClassifier]:
    Duongpred = cls.predict(DuongX)
    yscore = cls.score(DuongX, Duongy)
    print(classification_report(Duongy, Duongpred))
    print(cls.__class__.__name__, yscore)

              precision    recall  f1-score   support

           0       1.00      0.84      0.91       703
           1       0.00      0.00      0.00         0

    accuracy                           0.84       703
   macro avg       0.50      0.42      0.46       703
weighted avg       1.00      0.84      0.91       703

SVC 0.8378378378378378
              precision    recall  f1-score   support

           0       1.00      0.79      0.88       703
           1       0.00      0.00      0.00         0

    accuracy                           0.79       703
   macro avg       0.50      0.40      0.44       703
weighted avg       1.00      0.79      0.88       703

LinearSVC 0.7908961593172119
              precision    recall  f1-score   support

           0       1.00      0.73      0.85       703
           1       0.00      0.00      0.00         0

    accuracy                           0.73       703
   macro avg       0.50      0.37      0.42       703
weighted avg       1.0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
